In [11]:
from pathlib import Path
import simplekml
import pandas as pd
import xarray as xr
import numpy as np

### Data formating  
reformats the data in the handpicked coordinates, and concatinates old and new datasets, as well as generating basic files for viewing (kml, csv, etc)

In [12]:
kml = simplekml.Kml()
mounts = pd.read_csv('out/handpick_train_coords.csv')
lls = mounts.to_numpy()
for ind, ll in enumerate(lls):
    kml.newpoint(coords=[(ll[2], ll[1])], name=ll[0])
kml.save('out/handpick_train_coords.kml')

In [13]:
from smount_predictors import SeamountHelp
new_coords = mounts
old_coords = pd.read_csv('data/all_training.xyhrdnc', sep=' ', names=['lon', 'lat', 'height', 'radius', 'dept', 'name', 'cat'])
old_coords = old_coords[['lat', 'lon', 'radius', 'name']]
points = SeamountHelp.readKMLbounds(Path('data/seamount_training_zone.kml'))
old_coords = old_coords[(old_coords['lat'] > points[0]) & (old_coords['lat'] < points[1]) & (old_coords['lon'] > points[2]) & (old_coords['lon'] < points[3])]
train_coords = pd.concat([old_coords, new_coords], ignore_index=True)
train_coords.to_csv('out/train_mounts.csv')

### Training data generation
takes the new merged datasets and the vgg data and lables the vgg data, before outputing a new training dataset

In [14]:
data = xr.open_dataset('data/figure_datasets/raw_data.nc').to_dataframe().reset_index()
seamounts = train_coords
labled_data = SeamountHelp.seamount_radial_match(data, train_coords)

In [15]:
nc_data = xr.Dataset.from_dataframe(labled_data.set_index(['lat', 'lon']))
nc_data.to_netcdf('data/training_data_new.nc')